In [1]:
#Import Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import jellyfish
%matplotlib inline

In [2]:
#Read in data
names = ["fsid", "Full1"] + ['Source{}'.format(x) for x in range(1,21)]
df = pd.read_csv('/Users/jperryman/Desktop/BYU/Python/api_scrape.csv', names=names)

In [3]:
#Clean noise
df.Source1 = df.Source1.str.replace('( in[\s\S]+)',"")

In [4]:
#print(df.Source1)

In [5]:
df.Source1 = df.Source1.str.extract('([A-Z][ \w]+)',expand = False)

In [6]:
#print(df.Source3)

In [7]:
for x in ['Source{}'.format(x) for x in range(1,21)]:
    df[x] = df[x].str.replace('( in[\s\S]+)',"")
    df[x] = df[x].str.replace('Legacy NFS Source: ',"")
    df[x] = df[x].str.replace('[Ff]ind [Aa] [Gg]rave',"")
    df[x] = df[x].str.replace("\d","")
    df[x] = df[x].str.replace("\.","")
    df[x] = df[x].str.replace('([Tt]he )?United States (Federal )?Census',"")
    df[x] = df[x].str.replace('( the )',"")
    df[x] = df[x].str.extract('([A-Z][ \w]+)',expand = False)

In [8]:
#df.head()

In [9]:
df = df.set_index(['fsid', 'Full1'])
df = df.stack().reset_index()

In [10]:
#df.head()

In [12]:
for x in ['Full1'.format(x)]:
    df[x] = df[x].str.replace('[\'\"]\w+[\'\"]',"")
    df[x] = df[x].str.replace('\"',"")
    df[x] = df[x].str.replace("\'","")
    df[x] = df[x].str.replace('[\'\"]\w+[\'\"]',"")
    df[x] = df[x].str.replace('( [Jj]r)',"")
    df[x] = df[x].str.replace('( [Ss]r)',"")
    df[x] = df[x].str.replace('\.',"")

In [13]:
del df['level_2']

In [14]:
#df.head()

In [15]:
df.columns = ['fsid', 'Full1', 'Full2']

In [16]:
for x in ['Full2'.format(x)]:
    df[x] = df[x].str.replace('\"',"")
    df[x] = df[x].str.replace('[\'\"]\w+[\'\"]',"")
    df[x] = df[x].str.replace("\'","")
    df[x] = df[x].str.replace('( [Jj]r)',"")
    df[x] = df[x].str.replace('( [Ss]r)',"")
    df[x] = df[x].str.replace('\.',"")

In [17]:
#df.head()

In [18]:
df['score'] = df.apply(lambda row: jellyfish.jaro_distance(row['Full1'], row['Full2']), axis=1)

In [19]:
#df.head()

In [20]:
df = df[df.score != 1]
df = df[df.score >= .55]

In [21]:
#df.head()
df['Full1'] = df['Full1'].str.strip()
df['Full2'] = df['Full2'].str.strip()

In [22]:
del df['score']
df['First1'] = df.Full1
df['Mid1'] = df.Full1
df['Last1'] = df.Full1
df['First2'] = df.Full2
df['Mid2'] = df.Full2
df['Last2'] = df.Full2

In [23]:
cols = list(df)
cols.insert(5, cols.pop(cols.index('Full2')))
cols

['fsid',
 'Full1',
 'First1',
 'Mid1',
 'Last1',
 'Full2',
 'First2',
 'Mid2',
 'Last2']

In [24]:
df = df.loc[:, cols]

In [25]:
#df.head()

In [26]:
for x in ['First1'.format(x)]:
    #df[x] = df[x].str.replace('( )\w+',"")
    df[x] = df[x].str.extract('(^\w+)', expand=False)
    #df[x] = df[x].str.replace('^\w+',"")

In [27]:
#df.head()

In [28]:
for x in ['Mid1'.format(x)]:
    df[x] = df[x].str.replace('^\w+',"")
    df[x] = df[x].str.replace('\w+$',"")
    #df[x] = df[x].str.extract('( \w+ )', expand=False)
    #df[x] = df[x].str.replace('(\w+$)',"")
    #df[x] = df[x].str.replace('(^\w+)',"")

In [29]:
for x in ['Last1'.format(x)]:
    #df[x] = df[x].str.replace('\w+( )',"")
    df[x] = df[x].str.extract('(\w+$)', expand=False)
    #df[x] = df[x].str.replace('\w+$',"")

In [30]:
##NAME 1 complete
df.head()

,fsid,Full1,First1,Mid1,Last1,Full2,First2,Mid2,Last2
0,LDBJ-136,Catherine Englehart,Catherine,,Englehart,Catharine Englehart,Catharine Englehart,Catharine Englehart,Catharine Englehart
1,LDBJ-136,Catherine Englehart,Catherine,,Englehart,Katie Englehart,Katie Englehart,Katie Englehart,Katie Englehart
2,LDBJ-136,Catherine Englehart,Catherine,,Englehart,Kate Wagner,Kate Wagner,Kate Wagner,Kate Wagner
3,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Philip E Fuller,Philip E Fuller,Philip E Fuller,Philip E Fuller
4,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Phillip E Fuller,Phillip E Fuller,Phillip E Fuller,Phillip E Fuller


In [31]:
for x in ['First2'.format(x)]:
    #df[x] = df[x].str.replace('( )\w+',"")
    df[x] = df[x].str.extract('(^\w+)', expand=False)
    #If only one name assume it is first name

In [32]:
for x in ['Mid2'.format(x)]:
    #df[x] = df[x].str.extract('( \w+ )', expand=False)
    df[x] = df[x].str.replace('(\w+$)',"")
    df[x] = df[x].str.replace('(^\w+)',"")
    #KEEPS MIDDLE INITIAL  df[x] = df[x].str.replace('([A-Z])\w+',"")

In [33]:
for x in ['Last2'.format(x)]:
    #df[x] = df[x].str.replace('\w+( )',"")
    df[x] = df[x].str.extract('(\w+$)', expand=False)

In [34]:
df.head()

,fsid,Full1,First1,Mid1,Last1,Full2,First2,Mid2,Last2
0,LDBJ-136,Catherine Englehart,Catherine,,Englehart,Catharine Englehart,Catharine,,Englehart
1,LDBJ-136,Catherine Englehart,Catherine,,Englehart,Katie Englehart,Katie,,Englehart
2,LDBJ-136,Catherine Englehart,Catherine,,Englehart,Kate Wagner,Kate,,Wagner
3,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Philip E Fuller,Philip,E,Fuller
4,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Phillip E Fuller,Phillip,E,Fuller


In [35]:
df['Match'] = 1
df = df.fillna('')

In [36]:
df.head(50)

,fsid,Full1,First1,Mid1,Last1,Full2,First2,Mid2,Last2,Match
0,LDBJ-136,Catherine Englehart,Catherine,,Englehart,Catharine Englehart,Catharine,,Englehart,1
1,LDBJ-136,Catherine Englehart,Catherine,,Englehart,Katie Englehart,Katie,,Englehart,1
2,LDBJ-136,Catherine Englehart,Catherine,,Englehart,Kate Wagner,Kate,,Wagner,1
3,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Philip E Fuller,Philip,E,Fuller,1
4,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Phillip E Fuller,Phillip,E,Fuller,1
5,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Philip E Fuller,Philip,E,Fuller,1
8,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Philip Fuller,Philip,,Fuller,1
11,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Phillip E Fuller,Phillip,E,Fuller,1
12,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Philip E Fuller,Philip,E,Fuller,1
13,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Phillip E Fuller,Phillip,E,Fuller,1


In [37]:
df.loc[df.First2 == df.Last2, "Last2"] = ""

In [38]:
df.head(1000)

,fsid,Full1,First1,Mid1,Last1,Full2,First2,Mid2,Last2,Match
0,LDBJ-136,Catherine Englehart,Catherine,,Englehart,Catharine Englehart,Catharine,,Englehart,1
1,LDBJ-136,Catherine Englehart,Catherine,,Englehart,Katie Englehart,Katie,,Englehart,1
2,LDBJ-136,Catherine Englehart,Catherine,,Englehart,Kate Wagner,Kate,,Wagner,1
3,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Philip E Fuller,Philip,E,Fuller,1
4,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Phillip E Fuller,Phillip,E,Fuller,1
5,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Philip E Fuller,Philip,E,Fuller,1
8,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Philip Fuller,Philip,,Fuller,1
11,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Phillip E Fuller,Phillip,E,Fuller,1
12,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Philip E Fuller,Philip,E,Fuller,1
13,LR4C-C64,Philip Ely Fuller,Philip,Ely,Fuller,Phillip E Fuller,Phillip,E,Fuller,1
